Importing the required libraries

In [29]:
import cv2
import os
import numpy as np
from pathlib import Path


In [30]:
# Creating a seperate  output directory as Detected_videos
output_folder = 'Detected_videos'
os.makedirs(output_folder, exist_ok=True)

# Initializing  the  cat face detector.....

cat_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalcatface_extended.xml')

def detect_objects(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(
        f'{output_folder}/detected_{os.path.basename(video_path)}',
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (frame_width, frame_height)
    )

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Object detection in the cat  video.mp4
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cats = cat_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        for (x,y,w,h) in cats:
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

        out.write(frame)

    cap.release()
    out.release()


In [31]:

###segmentation

def segment_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fgbg = cv2.createBackgroundSubtractorMOG2()
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Initialize out within segment_video function
    out = cv2.VideoWriter(
        f'{output_folder}/segmented_{os.path.basename(video_path)}', # Changed output file name
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (frame_width, frame_height)
    )

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Create segmentation mask
        fg_mask = fgbg.apply(frame)
        segmented = cv2.bitwise_and(frame, frame, mask=fg_mask)

        # Apply processing to segmented frame
        gray_segmented = cv2.cvtColor(segmented, cv2.COLOR_BGR2GRAY)
        cats_segmented = cat_cascade.detectMultiScale(gray_segmented, scaleFactor=1.1, minNeighbors=5)

        for (x,y,w,h) in cats_segmented:
            cv2.rectangle(segmented, (x,y), (x+w,y+h), (0,255,0), 2)

        out.write(segmented)


    cap.release()
    out.release()



In [32]:
video_files = ['cat_7.mp4', 'cat_8.mp4', 'cat_9.mp4']

for video in video_files:
    detect_objects(video)

KeyboardInterrupt: 